# Population 
Ces données sont disponible avec l'API de l'Insee : 
https://catalogue-donnees.insee.fr/fr/explorateur/DS_RP_POPULATION_PRINC

In [1]:
import requests
import os
import zipfile

# Définir l'identifiant du jeu de données
dataset_id = "DS_RP_POPULATION_PRINC"

def download_and_extract_dataset(dataset_id, output_dir=None):
    """
    Télécharge le fichier ZIP d'un jeu de données Melodi et l'extrait
    
    Args:
        dataset_id: Identifiant du jeu de données
        output_dir: Dossier où extraire les fichiers (optionnel)
        
    Returns:
        Le chemin du dossier contenant les fichiers extraits
    """
    # URL pour le téléchargement du ZIP
    zip_url = f"https://api.insee.fr/melodi/file/{dataset_id}/{dataset_id}_CSV_FR"
    
    # Si aucun dossier de sortie n'est spécifié, utiliser le répertoire courant avec l'identifiant
    if output_dir is None:
        output_dir = dataset_id
    
    # Chemin du fichier ZIP temporaire
    temp_zip_path = f"{dataset_id}.zip"
    
    print(f"Téléchargement du fichier ZIP depuis {zip_url}")
    
    try:
        # Télécharger le fichier ZIP par morceaux pour économiser la mémoire
        with requests.get(zip_url, stream=True, verify=True) as response:
            response.raise_for_status()  # Lever une exception si la requête échoue
            
            # Obtenir la taille du fichier si disponible
            total_size = int(response.headers.get('content-length', 0))
            total_size_mb = total_size / (1024 * 1024)
            
            if total_size > 0:
                print(f"Taille du fichier: {total_size_mb:.2f} Mo")
            
            # Enregistrer le fichier par morceaux
            with open(temp_zip_path, 'wb') as f:
                downloaded = 0
                chunk_size = 8192  # 8 Ko par morceau
                
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:  # Filtrer les keep-alive chunks
                        f.write(chunk)
                        downloaded += len(chunk)
                        
                        # Afficher la progression environ tous les Mo
                        if total_size > 0 and downloaded % (1024 * 1024) < chunk_size:
                            progress = (downloaded / total_size) * 100
                            print(f"Progression: {progress:.1f}% ({downloaded/(1024*1024):.2f} Mo / {total_size_mb:.2f} Mo)")
        
        print(f"Téléchargement terminé avec succès!")
        
        # Créer le dossier de sortie s'il n'existe pas
        os.makedirs(output_dir, exist_ok=True)
        
        # Extraire le contenu du ZIP
        print(f"Extraction du contenu dans le dossier {output_dir}...")
        with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
        
        # Compter le nombre de fichiers extraits
        file_count = 0
        for _, _, files in os.walk(output_dir):
            file_count += len(files)
        
        print(f"Extraction terminée avec succès! {file_count} fichiers extraits dans {output_dir}")
        
        # Lister les fichiers principaux
        main_files = [f for f in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, f))]
        if main_files:
            print("Fichiers principaux extraits:")
            for file in main_files:
                file_size = os.path.getsize(os.path.join(output_dir, file)) / (1024 * 1024)  # Taille en Mo
                print(f"  - {file} ({file_size:.2f} Mo)")
        
        # Demander à l'utilisateur s'il souhaite supprimer le fichier ZIP temporaire
        delete_zip = input("Souhaitez-vous supprimer le fichier ZIP téléchargé ? (o/n): ")
        if delete_zip.lower() in ['o', 'oui', 'y', 'yes']:
            os.remove(temp_zip_path)
            print(f"Fichier ZIP {temp_zip_path} supprimé.")
        else:
            print(f"Le fichier ZIP {temp_zip_path} a été conservé.")
        
        return output_dir
        
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors du téléchargement: {e}")
        return None
    except zipfile.BadZipFile:
        print(f"Erreur: Le fichier téléchargé n'est pas un fichier ZIP valide.")
        return None
    except Exception as e:
        print(f"Erreur inattendue: {e}")
        return None

# Exécuter le téléchargement et l'extraction
if __name__ == "__main__":
    extract_dir = download_and_extract_dataset(dataset_id)
    
    if extract_dir:
        print(f"\nLes données sont maintenant disponibles dans le dossier: {extract_dir}")
        print("Vous pouvez utiliser ces fichiers pour vos analyses.")

Téléchargement du fichier ZIP depuis https://api.insee.fr/melodi/file/DS_RP_POPULATION_PRINC/DS_RP_POPULATION_PRINC_CSV_FR
Taille du fichier: 27.49 Mo
Progression: 3.6% (1.00 Mo / 27.49 Mo)
Progression: 7.3% (2.00 Mo / 27.49 Mo)
Progression: 10.9% (3.00 Mo / 27.49 Mo)
Progression: 14.5% (4.00 Mo / 27.49 Mo)
Progression: 18.2% (5.00 Mo / 27.49 Mo)
Progression: 21.8% (6.00 Mo / 27.49 Mo)
Progression: 25.5% (7.00 Mo / 27.49 Mo)
Progression: 29.1% (8.00 Mo / 27.49 Mo)
Progression: 32.7% (9.00 Mo / 27.49 Mo)
Progression: 36.4% (10.00 Mo / 27.49 Mo)
Progression: 40.0% (11.00 Mo / 27.49 Mo)
Progression: 43.6% (12.00 Mo / 27.49 Mo)
Progression: 47.3% (13.00 Mo / 27.49 Mo)
Progression: 50.9% (14.00 Mo / 27.49 Mo)
Progression: 54.6% (15.00 Mo / 27.49 Mo)
Progression: 58.2% (16.00 Mo / 27.49 Mo)
Progression: 61.8% (17.00 Mo / 27.49 Mo)
Progression: 65.5% (18.00 Mo / 27.49 Mo)
Progression: 69.1% (19.00 Mo / 27.49 Mo)
Progression: 72.7% (20.00 Mo / 27.49 Mo)
Progression: 76.4% (21.00 Mo / 27.49 Mo)

documentation de l'API : https://portail-api.insee.fr/catalog/api/a890b735-159c-4c91-90b7-35159c7c9126/doc?page=712a242a-9d5c-4675-aa24-2a9d5c4675c4